※このノートブックはGoogle Colabでの実行を前提としています。

# Step52~54 GPU, モデルの保存・読込み, Dropoutとテストモード

# Step 52 GPU

全員がローカル環境でGPUを使えるとは限らないのでここではGoogle Colab上での実行を前提とします。
※テキスト曰くローカルでGPUを使用するにはNVIDIAのGPUが必要だそうです。

まずは先に完成形をみてましょう。

## DeZeroのインストール

まずはDeZeroをインストールします。DeZeroは[PyPI](https://pypi.org/project/dezero/)に登録してあるので、コマンドの`pip install dezero`からインストールすることができます。

In [1]:
!pip install dezero

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


続いて、DeZeroでGPUが使えるかをチェックしてみます。

In [2]:
import dezero
dezero.cuda.gpu_enable

True

`True`であればGPUが使える状態です。先に進みましょう。

`False`の場合は、Google ColabのGPU設定が必要になります。それは下記の手順で行います。

* メニューの「ランタイム」から「ランタイムのタイプを変更」を選択
* 「ハードウェアアクセラレータ」のドロップメニューから「GPU」を選択

## Train MNIST on GPU
ではGPUを使ってDeZeroでMNISTの学習を行ってみます。

In [3]:
import time
import dezero
import dezero.functions as F
from dezero import optimizers
from dezero import DataLoader
from dezero.models import MLP

max_epoch = 5
batch_size = 100
gpu_times = []

train_set = dezero.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
# GPU mode
train_loader.to_gpu()
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)
model.to_gpu()

for epoch in range(max_epoch):
    start = time.time()
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    elapsed_time = time.time() - start
    gpu_times.append(elapsed_time)
    print('epoch: {}, loss: {:.4f}, time: {:.4f}[sec]'.format(
        epoch + 1, sum_loss / len(train_set), elapsed_time))

Downloading: train-images-idx3-ubyte.gz
[##############################] 100.00% Done
Downloading: train-labels-idx1-ubyte.gz
[##############################] 100.00% Done
epoch: 1, loss: 1.9313, time: 16.9433[sec]
epoch: 2, loss: 1.2998, time: 3.2400[sec]
epoch: 3, loss: 0.9331, time: 3.3411[sec]
epoch: 4, loss: 0.7444, time: 3.2382[sec]
epoch: 5, loss: 0.6376, time: 3.2588[sec]


## Train MNIST on CPU
比較としてCPUでも学習を行い、学習速度を比較してみましょう。

In [4]:

cpu_times = []

train_set = dezero.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)

for epoch in range(max_epoch):
    start = time.time()
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    elapsed_time = time.time() - start
    cpu_times.append(elapsed_time)
    print('epoch: {}, loss: {:.4f}, time: {:.4f}[sec]'.format(
        epoch + 1, sum_loss / len(train_set), elapsed_time))

epoch: 1, loss: 1.9014, time: 14.1629[sec]
epoch: 2, loss: 1.2741, time: 11.5127[sec]
epoch: 3, loss: 0.9175, time: 10.4902[sec]
epoch: 4, loss: 0.7353, time: 10.5452[sec]
epoch: 5, loss: 0.6322, time: 10.6912[sec]


以上より、結果は次のようになります。

In [5]:
cpu_avg_time = sum(cpu_times) / len(cpu_times)
gpu_avg_time = sum(gpu_times) / len(gpu_times)

print('CPU: {:.2f}[sec]'.format(cpu_avg_time))
print('GPU: {:.2f}[sec]'.format(gpu_avg_time))
print('GPU speedup over CPU: {:.1f}x'.format(cpu_avg_time/gpu_avg_time))

CPU: 11.48[sec]
GPU: 6.00[sec]
GPU speedup over CPU: 1.9x


ここからは上記のようなGPU対応フレームワークにDeZeroをアップデートしていきます。

## CuPy

GPUで並列計算を行うためのライブラリであるCuPyを使ってDeZeroをGPU対応化します。<br>
公式ドキュメント：https://docs.cupy.dev/en/stable/reference/ndarray.html

まずはインストールします。

In [6]:
# Google Colabではデフォルトでインストール済みなので不要
# !pip install cupy

# 逆にCuPyが入っていない場合を再現するときはアンインストール
# !pip uninstall cupy

CuPyの特徴は**Numpyと共通のAPIをもつ**ことです。
Numpyと同じような操作を行ってみましょう。

In [7]:
import cupy as cp

# 配列データとその操作
x = cp.arange(6)
print('x')
print(x)
print('\n')

print('x.shape')
print(x.shape)
print('\n')

print('x.reshape(2,3)')
print(x.reshape(2,3))
print('\n')

print('x.ndim')
print(x.ndim)
print('\n')

# スカラー
s = cp.array(2)
print('s')
print(s)
print('\n')

# ブロードキャスト
y = x + s
print('x + s')
print(y)
print('\n')


x
[0 1 2 3 4 5]


x.shape
(6,)


x.reshape(2,3)
[[0 1 2]
 [3 4 5]]


x.ndim
1


s
2


x + s
[2 3 4 5 6 7]




GPU対応化させると言っても、すごく大雑把に言えばNumpyをCuPyに書き換えれば並列計算に対応した配列を使うことが出来ます。<br>
※今回Numpyの型で定義した変数をCuPyの型に書き換える作業は省略します。実はそこまで機械的に書き換えることが出来ることを後半で解説します。


ただし、NumpyをCuPyに置き換えてしまうとGPU上での実行が前提となってしまうため、DeZeroではNumpy/CuPyを切り替える仕組みを作成します。

### Numpy/CuPyを切り替える仕組みづくり

実はCuPyにはNumpy⇒CuPy、CuPy⇒Numpyを変換する関数が用意されています。<br>
これを使えは容易に両者を切り替えることが出来ます。

ちなみにこの切り替えを行うたびにCPUメモリとGPUメモリの間でデータ転送が行われます。<br>
この転送処理がボトルネックになりやすいのでなるべく回数を少なくするように実装したほうが良いです。

In [8]:
import numpy as np
import cupy as cp

# Numpy ⇒ CuPy
n = np.array(1.0)
c = cp.asarray(n)  # Numpy⇒CuPy変換
assert type(c) == cp.ndarray  # assertition errorにならなければCuPyの配列

# CuPy ⇒ Numpy
m = cp.asnumpy(c)  # CuPy⇒Numpy変換
assert type(m) == np.ndarray  # assertition errorにならなければNumpyの配列



また、 cp.get_array_moduleという関数を使えばNumpy/CuPyの両方のデータ型を入力として受け付けることが出来ます。

get_array_moduleはNumpy型の変数を受け取れば**Numpyモジュール**を返し、CuPy型の
変数を受け取れば**CuPyモジュール**を返します。<br>

In [9]:

# Numpyを入力に受け取る
x = np.array([1, 2, 3])
xp = cp.get_array_module(x)
assert xp == np
print(type(xp))

# CuPyを入力に受け取る
x = cp.array([1, 2, 3])
xp = cp.get_array_module(x)
assert xp == cp
print(type(xp))


<class 'module'>
<class 'module'>


DeZeroではCuPy関連のモジュールをdezero/cuda.pyにまとめます。<br>
(CUDAとはNVIDIAが提供するGPU向け開発環境の名前)

今回は完成済みのDeZeroをインポートしているのでポイントを眺めるだけにします。

まずはNumpyとCuPyをインポートします。

DeZeroでは、実行環境にCuPyがインストールされていることを**前提にはしません**<br>
そこでCuPyがインストールされていない場合にもエラーにならないようにしています。

In [10]:
import numpy as np

# GPU使用フラグ
gpu_enable = True

try:
    # CuPyインポート
  import cupy as cp
  cupy = cp
except ImportError:
    # インポートエラーの場合
  gpu_enable = False  # GPU使用フラグ：オフ
  
from dezero import Variable

次に先ほどみたCuPyの関数をDeZero用のラッパー関数として作成し直します。<br>
オリジナルとの相違点はCuPyがインストールされていない場合の挙動を追加している点です。<br>
※もちろんCuPyがインストールされていない場合の考慮をCuPyの中ではできません。そこでラッパー関数で考慮します。

In [11]:
# cp.get_array_moduleのラッパー
def get_array_module(x):
    # Variable型の場合
    if isinstance(x, Variable):
        # 中身を取り出し
        x = x.data
    
    # GPUモードではない場合
    if not gpu_enable:
        # Numpyモジュールを返す
        return np
    # GPUモードの場合Numpy/CuPyのうち適切な方を返す
    xp = cp.get_array_module(x)
    return xp

# cp.asnumpyのラッパー
def as_numpy(x):
    # Variable型の場合
    if isinstance(x, Variable):
        # 中身を取り出し
        x = x.data
    
    # スカラーの場合
    if np.isscalar(x):
        # arrayに包んで返す
        return np.array(x)
    # np.ndarrayの場合
    elif isinstance(x, np.ndarray):
        # そのまま返す
        return x
    # cp.ndarrayの場合、np.ndarrayに変換して返す
    return cp.asnumpy(x)

# cp.asarrayのラッパー
def as_cupy(x):
    # Variable型の場合
    if isinstance(x, Variable):
        # 中身を取り出し
        x = x.data
    
    # GPUモードではない場合
    if not gpu_enable:
        # 例外にスロー
        raise Exception('CuPy cannot be loaded. Install CuPy!')

    # GPUモードの場合、cp.ndarrayに変換して返す    
    return cp.asarray(x)


## 既存クラスのGPU対応

ここではVariable / Layer / DataLoader クラスをGPUに対応させるための追加実装を行います。

### VariableクラスのGPU対応

まずはVariableクラスの定義の前に`array_types`というtuple型の変数を追加します。
この変数はarrayとして使用出来る型を保持します。

In [12]:
try:
    # CuPyのインポートに成功した場合
    import cupy
    array_types = (np.ndarray, cupy.ndarray)
except ImportError:
    # CuPyのインポートに失敗した場合
    array_types = (np.ndarray)


ではVariableクラスを書き換えていきます。変更点は3つです。

1．\_\_init\_\_メソッドでは最初の型チェックをarray_typesで行うようにします。

2．backwardの冒頭でself.dataの型（言い換えるとGPU使用有無）に応じてNumpy/CuPyを切り替えるようにします。

3．CPUとGPUの間でデータ転送を行うための関数を追加します。実体としては先ほど作成したas_numpyとas_cupyです。

In [13]:
class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, array_types):
                raise TypeError('{} is not supported'.format(type(data)))
        ...  # 以降変更なし
    
    def backward(self, retain_grad=False, create_graph=False):
        if self.grad is None:
            xp = dezero.cuda.get_array_module(self.data)
            self.grad = Variable(xp.ones_like(self.data))
        ...  # 以降変更なし

    def to_cpu(self):
        if self.data is not None:
            self.data = dezero.cuda.as_numpy(self.data)

    def to_gpu(self):
        if self.data is not None:
            self.data = dezero.cuda.as_cupy(self.data)


### LayerクラスのGPU対応

Layerクラスでの変更点はGPU/CPUの使用状況に応じてパラメータをCPU/GPUに転送するメソッドを追加します。
実装的には直前で作成したVariableクラスの同名のメソッドを呼び出しています。

In [14]:
class Layer:
    ...
    def to_cpu(self):
        for param in self.params():
            param.to_cpu()
    def to_gpu(self):
        for param in self.params():
            param.to_gpu()


### DataLoaderクラスのGPU対応

DataLoaderクラスはGPUの使用有無をユーザが選択できるようにします。<br>
そのために`gpu`という変数を追加し、この変数によってGPUの使用有無をコントロールします。

`to_gpu`と`to_cpu`メソッドでこの変数を書き換えてモードを切り替えます。

\_\_next\_\_メソッドでarrayを生成する際に`gpu`をみてNumpy/CuPyどちらを使用するか選択します。<br>
ここでは`get_array_module`関数を使わずモジュールを選択しています。（演習問題：それはなぜでしょう？）

In [15]:
import numpy as np
from dezero import cuda

class DataLoader:
    def __init__(self, dataset, batch_size, shuffle=True, gpu=False ):
        ...
        self.gpu = gpu
        ...

    def __next__(self):
        ...
        xp = cuda.cupy if self.gpu else np
        x = xp.array([example[0] for example in batch])
        t = xp.array([example[1] for example in batch])
        ...

    def to_cpu(self):
        self.gpu = False

    def to_gpu(self):
        self.gpu = True

## 関数のGPU対応

関数をGPU対応にするには、Numpyの使用箇所をNumpy/CuPyを切り替えられるように変更する必要があります。<br>
sin関数を例に修正を行ってみます。

修正箇所はforwardでnp.sinを使用していた箇所に`cuda.get_array_module`を使って適切なモジュールを選択できるようにし、そのモジュールのsinを呼び出すようにしています。

このような修正をすべての関数で行います。（量はありますが機械的に置き換えることが出来ます）

```python
from dezero import cuda

class Sin(Function):
    def forward(self, x):
        xp = cuda.get_array_module(x)
        y = xp.sin(x)
        return y
    def backward(self, gy):
        x, = self.inputs
        gx = gy * cos(x)
        return gx
```

## 四則演算のGPU対応

四則演算をGPU対応に修正していきます。

まずは`as_array`をNumpy/CuPyどちらのarrayに対応させます。


次に各演算の関数を修正します。

In [16]:
def as_array(x, array_module=np):
    if np.isscalar(x):
        return array_module.array(x)
    return x

ここでは足し算を修正します。他の演算のは同様に修正すればよいのでここでは省略します。

計算を定義しているAddクラスの修正はありません。<br>
（NumpyとCuPyのAPIで共通仕様となっているメソッドを使用してるからです）

add関数の中で使用しているas_array関数をGPU対応のas_array関数に合わせて引数を渡し直す必要があります。<br>
x0.dataのデータ型によって適切なモジュールの配列に変換するようにします。

```python
class Add(Function):
    def forward(self, x0, x1):
        self.x0_shape, self.x1_shape = x0.shape, x1.shape
        y = x0 + x1
        return y

    def backward(self, gy):
        gx0, gx1 = gy, gy
        if self.x0_shape != self.x1_shape:  # for broadcaset
            gx0 = dezero.functions.sum_to(gx0, self.x0_shape)
            gx1 = dezero.functions.sum_to(gx1, self.x1_shape)
        return gx0, gx1


def add(x0, x1):
    x1 = as_array(x1, dezero.cuda.get_array_module(x0.data))
    return Add()(x0, x1)

```

以上でGPU対応は終了です。

これで本セクション冒頭のMNISTのGPU対応版のコードを実行できるようになります。

# Step 53 モデルの保存・読込み

モデルの保存と読込み機能を実装します。<br>
より具体的にはモデルのパラメータの保存と読込みを行えるようにします。
パラメータはNumpyまたはCuPyのndarrayインスタンスなので、これを保存と読込みできるようにします。

Numpyにはndarrayの保存と読込み機能ががあります。<br>
これを利用していきます。

※CuPyにもndarrayインスタンスを保存と読込みする機能があるようだが、DeZeroではNumpyに寄せて使うらしい。（np.ndarrayで保存したものをcp.ndarrayで読込んだりすると何か不都合がある？）

## np.ndarrayの保存と読込み

np.ndarrayを一つ保存と読込みを行うにはsave関数とload関数を使います。

なお、拡張子は`npy`を指定する必要があります。

In [17]:
import numpy as np

# np.ndarrayインスタンス生成
x = np.array([1, 2, 3])

# 保存
np.save('test.npy', x)

# 読込み
y = np.load('test.npy')

print(y)

[1 2 3]


次に複数のnp.ndarrayを保存と読込みする方法です。<br>
この場合はsave**z**関数とload関数を使用します。

保存時にキーワード引数を指定することが出来ます。この場合読込み時にキー情報を指定することで特定のndarrayを取り出すことが出来ます。

In [18]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

np.savez('test.npz', x1=a, x2=b)

arrays = np.load('test.npz')

x1 = arrays['x1']
x2 = arrays['x2']

print(x1)
print(x2)


[1 2 3]
[4 5 6]


また、次のようにキーワード引数をディクショナリにまとめて渡すこともできます。<br>
その場合は渡す先に"**"をつけてディクショナリの中身が展開されるようにします。

In [19]:
x1 = np.array([1, 2, 3])
x2 = np.array([4, 5, 6])
data = {'x1':x1, 'x2':x2}

np.savez('test.npz', **data)

arrays = np.load('test.npz')

x1 = arrays['x1']
x2 = arrays['x2']

print(x1)
print(x2)


[1 2 3]
[4 5 6]


ちなみにnpy、npzはバイナリ形式でndarrayの中身を保存します。<br>
試しに中身をみるとこんな感じ

In [20]:
with open('./test.npz', 'rb') as f:
    s = f.read()
    print(s)

b"PK\x03\x04\x14\x00\x00\x00\x00\x00\x00\x00!\x00\xbc\x91U\x8e\x98\x00\x00\x00\x98\x00\x00\x00\x06\x00\x14\x00x1.npy\x01\x00\x10\x00\x98\x00\x00\x00\x00\x00\x00\x00\x98\x00\x00\x00\x00\x00\x00\x00\x93NUMPY\x01\x00v\x00{'descr': '<i8', 'fortran_order': False, 'shape': (3,), }                                                            \n\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00PK\x03\x04\x14\x00\x00\x00\x00\x00\x00\x00!\x00\xd5\x9co\xd3\x98\x00\x00\x00\x98\x00\x00\x00\x06\x00\x14\x00x2.npy\x01\x00\x10\x00\x98\x00\x00\x00\x00\x00\x00\x00\x98\x00\x00\x00\x00\x00\x00\x00\x93NUMPY\x01\x00v\x00{'descr': '<i8', 'fortran_order': False, 'shape': (3,), }                                                            \n\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00PK\x01\x02\x14\x03\x14\x00\x00\x00\x00\x00\x00\x00!\x00\xbc\x91U\x8e\x98\x00\x00\x00\x98\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x00

また、savezは非圧縮でパラメータを保存します。

圧縮して保存したい場合はnp.savez_compressed関数を使います。DeZeroではこちらを使うことにします。

### パラメータの保存・読込み機能の作成

DeZeroではLayerクラスが入れ子構造をとることが出来ました。<br>

```python
layer = Layer()

l1 = Layer()
l1.p1 = Parameter(np.array(1))

layer.l1 = l1
layer.p2 = Parameter(np.array(2))
layer.p3 = Parameter(np.array(3))
```

これを「1つのフラットはディクショナリ」に変換して保存・読込みを行うようにします。

In [21]:
class Layer:
    ...
    def _flatten_params(self, params_dict, parent_key=""):
        for name in self._params:
            # パラメータ取出し
            obj = self.__dict__[name]
            # キー情報作成
            key = parent_key + '/' + name if parent_key else name

            # Layerクラスの場合
            if isinstance(obj, Layer):
                # _flatten_params呼出し
                obj._flatten_params(params_dict, key)
            else:
                # パラメータ格納
                params_dict[key] = obj

`flatten_params`メソッドによって引数`params_dict`に渡したディクショナリにパラメータとキー情報のペアが登録されます。<br>
メソッドの中でさらに`_flatten_params`メソッド呼び出しており、入れ子になっているLayerクラスからもパラメータを取り出しています。

元の入れ子構造はキー情報名で表現しています。

In [22]:
from dezero import Layer, Parameter

layer = Layer()

l1 = Layer()
l1.p1 = Parameter(np.array(1))

layer.l1 = l1
layer.p2 = Parameter(np.array(2))
layer.p3 = Parameter(np.array(3))

params_dict = {}
layer._flatten_params(params_dict)
print(params_dict)

{'p3': variable(3), 'p2': variable(2), 'l1/p1': variable(1)}


これを使ってパラメータの保存用と読込み用のメソッドをLayerクラスに実装しましょう。

各行のうごきは以下のソースにコメントで記載しています。<br>
ポイントは`_flatten_params`を`save_weights`と`load_weights`の両方で呼び出していることです。<br>
これにより、保存時と読込み時で同じルールに従ってパラメータのキー名を作成して突合できる仕様になっています。（逆にパラメータやレイヤの`name`変数を書き換える場合は突合できなくなるので注意です）

```python

import os

class Layer:
    ...

    def save_weights(self, path):
        #GPUモード：オフ
        self.to_cpu()  # np.ndarrayに固定

        # パラメータ格納用dict初期化
        params_dict = {}
        # パラメータとキー情報を格納
        self._flatten_params(params_dict)
        # params_dictからキー情報とパラメータの中身をペアで取出し
        array_dict = {key: param.data for key, param in params_dict.items() if param is not None}
        # np.ndarrayの保存処理
        try:
            np.savez_compressed(path, **array_dict)
        # ユーザによる中断の場合
        except (Exception, KeyboardInterrupt) as e:
            # 作成されたファイルを削除
            if os.path.exists(path):
                os.remove(path)
            raise

    def load_weights(self, path):
        # np.ndarrayの読込み処理
        npz = np.load(path)

        # パラメータ格納用dict初期化
        params_dict = {}

        # パラメータとキー情報を格納
        self._flatten_params(params_dict)
        for key, param in params_dict.items():
            param.data = npz[key]  # 読込んだnp.ndarrayを各パラメータに格納

```

最後にこのメソッドを使ってみましょう。

MNISTの学習に保存・読込みも組み込むと以下になります。

In [23]:
import os
import dezero
import dezero.functions as F
from dezero import optimizers
from dezero import DataLoader
from dezero.models import MLP

max_epoch = 3
batch_size = 100

train_set = dezero.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)

model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)

# パラメータの読み込み
if os.path.exists('my_mlp.npz'):
    model.load_weights('my_mlp.npz')

for epoch in range(max_epoch):
    sum_loss = 0
    
    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)
    
    print('epoch: {}, loss: {:.4f}'.format(
        epoch + 1, sum_loss / len(train_set)))

model.save_weights('my_mlp.npz')

epoch: 1, loss: 1.9074
epoch: 2, loss: 1.2758
epoch: 3, loss: 0.9205


# Step 54 Dropoutとテストモード

DeZeroにDropoutレイヤを実装します。

Dropoutは訓練時とテスト時で挙動が異なるので両者を区別できる仕組みも作ります。

## Dropout

詳細の説明は実践機械学習やぜろつく1で解説済みなので省略します。

訓練時に隠れ層のニューロンをランダムに選んで信号の伝達を消去する層です。<br>
これによってアンサンブル学習に似たような効果が期待できます。



復習としてDropoutの実装の核となるアイデアを見ておきましょう。

In [24]:
import numpy as np

dropout_ratio = 0.6
x = np.ones(10)
print(x)

# 学習時
mask = np.random.rand(10) > dropout_ratio  # [0,1)の乱数を生成し、dropout_rateより大きいかどうかの真偽値をもつ配列を作成
print(mask)

y = x * mask  # Trueはそのまま、Falseは0になる
print(y)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ True  True False False  True  True False  True False  True]
[1. 1. 0. 0. 1. 1. 0. 1. 0. 1.]


テスト時にもアンサンブル学習のような挙動をするようにします。<br>
アンサンブル学習では個別のモデルが学習した結果を平均して最終的なモデルの出力値とします。<br>
これをDropoutで実現するには、すべてのニューロンを使い、出力値のスケールを1-Dropout_rateに合わせます。（これが平均をとることに対応します。）

実装で表現すると以下です。

In [25]:
# テスト時
scale = 1 - dropout_ratio
y = x * scale
print(y)

[0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4]


これが"通常の"Dropoutです。（Direct Dropoutと呼ばれています）<br>

それに対してDeZeroで実装するのはInverted Dropout（反転したDropout）です。<br>
多くのフレームワークではInverted Dropout方式が採用されているそうです。

### Inverted Dropout

Direct Dropoutとの違いはスケールを合わせるタイミングです。

Direct Dropoutではテスト時に行いますが、Inverted Dropoutでは学習時に行います。<br>
実装は以下のようになります。

In [26]:
# 学習時
scale = 1 - dropout_ratio
mask = np.random.rand(*x.shape) > dropout_ratio
y = x * mask / scale
print(y)

# テスト時
y = x
print(y)

[2.5 2.5 0.  0.  2.5 0.  0.  2.5 0.  0. ]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


テスト時には何行わなくて済むように、学習時の出力を1/（1-dropout_rate）倍しておきます。<br>
これによってテスト時の出力値が学習時の出力値の（1-dropout_rate）倍の関係になります。

言い換えると、学習時にdropoutによってニューロン数が減ってしまうことの帳尻合わせを学習時に完結させていると言えます。

Inverted Dropoutを採用するメリットは2つです。
1. テスト時の処理速度が（少しだけ）早くなる
2. 学習時に動的にdropout率を設定できる

2をDirect Dropoutで行おうとするとテスト時にどのscaleに調整すればよいのか分からなくなるので動的に設定することは難しいです。<br>
Inverted Dropoutではつじつま合わせが学習時に完結しているので動的に設定していっても問題ないのです。

## テストモードの追加

学習時とテスト時の挙動を切り替えられる機能を追加します。

Configクラスに`train`という変数を追加します。<br>
デフォルトでTrueとし、test_mode関数が呼ばれるとFalse（テストモード）になります。<br>
この関数はwith句と合わせて使うことで、with句の中はテストモード時の挙動になります。




```python

class Config:
    enable_backprop = True
    train = True

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    yield
    setattr(Config, name, old_value)

def test_mode():
    return using_config('train', False)

```

## Dropoutレイヤの実装

これまで整理した内容をもとにDropoutレイヤを作成します。<br>
実装にあたってはConfig.trainを参照してモードを切り替えるように実装します。

In [27]:
from dezero import as_variable

def dropout(x, dropout_ratio=0.5):
    x = as_variable(x)

    if dezero.Config.train:
        xp = cuda.get_array_module(x)
        mask = xp.random.rand(*x.shape) > dropout_ratio
        scale = xp.array(1.0 - dropout_ratio).astype(x.dtype)
        y = x * mask / scale
        return y
    else:
        return x

これで次のようにDropoutレイヤが使えるようになります。

In [28]:
import numpy as np
from dezero import test_mode
import dezero.functions as F

x = np.ones(5)
print(x)

# 学習時
y = F.dropout(x)
print(y)

# テスト時
with test_mode():
    y = F.dropout(x)
    print(y)

[1. 1. 1. 1. 1.]
variable([0. 2. 2. 0. 0.])
variable([1. 1. 1. 1. 1.])
